In [1]:
!rm -rf ./*

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump, load

# Создание датасетов

In [3]:
!mkdir df_1 df_2 df_3 df_4

In [4]:
def data_create(noise=False):

    # Генерация данных
    n_obs = 800
    gender = pd.Categorical(np.random.randint(0, 2, size=n_obs))
    age = np.random.randint(18, 41, size=n_obs)
    medal = np.random.randint(0, 14, size=n_obs)
    salary = np.round(2 * age + 8 * medal + 10 * np.random.randn(n_obs) + 30)

    # Создание датафреймов
    data = pd.DataFrame(
        {"пол": gender, "возраст": age, "наличие_медали": medal, "зарплата_тыс": salary}
    )
    # Создание датафрейма с шумом
    if noise:
        data["зарплата_тыс"] = data["зарплата_тыс"] + 20 * np.random.randn(n_obs)

    return data

In [5]:
df_1 = data_create()
df_2 = data_create()
df_3 = data_create()
df_4 = data_create(noise=True)

df_1.to_csv("df_1/df.сsv", index=False)
df_2.to_csv("df_2/df.сsv", index=False)
df_3.to_csv("df_3/df.сsv", index=False)
df_4.to_csv("df_4/df.сsv", index=False)

# Препроцессинг данных и обучение модели



In [6]:
def select_cat_features(data):
    return (
        data.drop(columns="зарплата_тыс")
        .select_dtypes(include=object)
        .columns.to_list()
    )


def select_num_features(data):
    return (
        data.drop(columns="зарплата_тыс")
        .select_dtypes(exclude=object)
        .columns.to_list()
    )

In [7]:
cat_features = select_cat_features(df_1)
num_features = select_num_features(df_1)

In [8]:
# Создание пайплайна для препроцессинга данных и выполнения прогноза
pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            (
                "encoder",
                OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse=False),
                cat_features,
            ),
            ("scaler", StandardScaler(), num_features),
        ]
    ),
    LinearRegression(),
)

In [9]:
X = df_1.drop(columns=["зарплата_тыс"])
y = df_1["зарплата_тыс"]

# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Размер тренировочного набора данных (X_train):", X_train.shape)
print("Размер тестового набора данных (X_test):", X_test.shape)
print("Размер тренировочного набора данных (y_train):", y_train.shape)
print("Размер тестового набора данных (y_test):", y_test.shape)

Размер тренировочного набора данных (X_train): (640, 3)
Размер тестового набора данных (X_test): (160, 3)
Размер тренировочного набора данных (y_train): (640,)
Размер тестового набора данных (y_test): (160,)


In [10]:
# Инициализация и обучение модели
pipeline.fit(X_train, y_train)

# Сохранение обученной модели
dump(pipeline, "pipeline.joblib")

['pipeline.joblib']

In [11]:
# Прогнозирование на тестовом наборе данных
y_pred_df1 = pipeline.predict(X_test)

# Вычисление средней абсолютной ошибки (MAE) и коэффициента детерминации(R^2)
MAE = mean_absolute_error(y_test, y_pred_df1)
r2 = r2_score(y_test, y_pred_df1)
print("Средняя абсолютная ошибка (MAE):", MAE)
print("Коэффициент детерминации (R^2):", r2)

Средняя абсолютная ошибка (MAE): 7.804905994946589
Коэффициент детерминации (R^2): 0.9186480356297752


# Создание файла для тестирования

In [12]:
%%writefile test.py
import numpy as np
import pandas as pd
import pytest
from joblib import load
from sklearn.metrics import mean_absolute_error, r2_score


@pytest.fixture()
def load_pipeline():
    pipeline = load("/content/pipeline.joblib")
    return pipeline


@pytest.fixture()
def load_data_and_get_pred(load_pipeline):
    data = pd.read_csv("df.сsv")
    X = data.drop(columns="зарплата_тыс")
    y = data["зарплата_тыс"]
    cat_features = X.select_dtypes(include="category").columns.to_list()
    num_features = X.select_dtypes(exclude=object).columns.to_list()
    y_pred = load_pipeline.predict(X)
    return y, y_pred


def test_mse(load_data_and_get_pred):
    y, y_pred = load_data_and_get_pred
    assert mean_absolute_error(y, y_pred) < 10


def test_r2(load_data_and_get_pred):
    y, y_pred = load_data_and_get_pred
    assert r2_score(y, y_pred) > 0.9


Writing test.py


# Тестирование данных в различных датафреймах

In [13]:
%cd df_1
!pytest -v /content/test.py
%cd /content

/content/df_1
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1
collected 2 items                                                                                  

../test.py::test_mse PASSED                                                                  [ 50%]
../test.py::test_r2 PASSED                                                                   [100%]

======================================== 2 passed in 2.07s =========================================
/content


In [14]:
%cd df_2
!pytest -v /content/test.py
%cd /content

/content/df_2
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1
collected 2 items                                                                                  

../test.py::test_mse PASSED                                                                  [ 50%]
../test.py::test_r2 PASSED                                                                   [100%]

======================================== 2 passed in 4.31s =========================================
/content


In [15]:
%cd df_3
!pytest -v /content/test.py
%cd /content

/content/df_3
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1
collected 2 items                                                                                  

../test.py::test_mse PASSED                                                                  [ 50%]
../test.py::test_r2 PASSED                                                                   [100%]

======================================== 2 passed in 3.50s =========================================
/content


In [16]:
%cd df_4
!pytest -v /content/test.py
%cd /content

/content/df_4
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1
collected 2 items                                                                                  

../test.py::test_mse FAILED                                                                  [ 50%]
../test.py::test_r2 FAILED                                                                   [100%]

============================================= FAILURES =============================================
_____________________________________________ test_mse _____________________________________________

load_data_and_get_pred = (0       87.944357
1      113.086686
2      147.116170
3      109.749447
4      121.726113
          ...    
795     7...43.61274373, 120.19277817, 119.54704272, 101.78508471,
       171.11475942, 162.02043719, 

Как можем заметить, датасеты 1-3 с "качественными" данными успешно прошли тестирование, датасет 4, содержащий шум, не прошел тестирование.